<a href="https://colab.research.google.com/github/ImNick23/Test/blob/master/Copy_of_recommend_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Import warnings
import warnings
warnings.filterwarnings('ignore')

In [0]:
#Importing numpy, pandas and matplotlib packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# importing  data
beer= pd.read_csv("https://raw.githubusercontent.com/ImNick23/file/master/beer_data.csv")
beer.head()

,beer_beerid,review_profilename,review_overall
0,48215,stcules,3.0
1,52159,oline73,3.0
2,52159,alpinebryant,3.0
3,52159,rawthar,4.0
4,52159,RangerClegg,3.5


**Data Preparation**

In [5]:
#Check shape and null values in data
print(beer.shape)
beer.isnull().sum()

(475984, 3)


beer_beerid             0
review_profilename    100
review_overall          0
dtype: int64

In [6]:
#Remove all null values present in data
beer = beer.dropna(how='any',axis=0)
print(beer.shape)
beer.isnull().sum()

(475884, 3)


beer_beerid           0
review_profilename    0
review_overall        0
dtype: int64

In [7]:
#checking number of unique values in df
beer.nunique()

beer_beerid           40304
review_profilename    22497
review_overall           10
dtype: int64

In [8]:
#Checking duplicate values for same profile with same beer id
beer.duplicated(subset=["beer_beerid","review_profilename"], keep='first').sum()

1422

In [9]:
#Removing duplicate values for same profile with same beer id
beer.drop_duplicates(subset = ["beer_beerid","review_profilename"], keep = 'first', inplace = True)
beer.duplicated(subset=["beer_beerid","review_profilename"], keep='first').sum()

0

In [10]:
#Checking shape again
print(beer.shape)
beer.head()

(474462, 3)


,beer_beerid,review_profilename,review_overall
0,48215,stcules,3.0
1,52159,oline73,3.0
2,52159,alpinebryant,3.0
3,52159,rawthar,4.0
4,52159,RangerClegg,3.5


In [12]:
#Removing data for all the beers which have less than 50 revivews
beer=beer.groupby('beer_beerid').filter(lambda x : len(x)>50)
beer.shape

(294197, 3)

**Data exploration**

In [19]:
#Check unique values and count of beers for each rating present
print(beer.nunique())
beer.groupby('review_overall').count()

beer_beerid            2028
review_profilename    19447
review_overall            9
dtype: int64


,beer_beerid,review_profilename
review_overall,,
1.0,1682,1682
1.5,1873,1873
2.0,5987,5987
2.5,9038,9038
3.0,26895,26895
3.5,51606,51606
4.0,110316,110316
4.5,66724,66724
5.0,20076,20076


In [97]:
#The average beer ratings
print(beer.groupby('beer_beerid').review_overall.mean())

beer_beerid
5        3.553097
6        3.711462
7        3.317073
10       3.883028
14       3.830508
           ...   
74272    3.648148
74323    3.920290
74390    3.718750
74986    4.217054
75086    3.660494
Name: review_overall, Length: 2028, dtype: float64


In [103]:
#The average user ratings
beer.groupby('review_profilename').review_overall.mean()

review_profilename
0110x011       4.333333
01Ryan10       5.000000
04101Brewer    4.000000
05Harley       4.116667
0beerguy0      4.166667
                 ...   
zymurgy4all    3.703704
zymurgywhiz    4.250000
zythus         5.000000
zyzygy         4.166667
zzajjber       5.000000
Name: review_overall, Length: 19447, dtype: float64

In [23]:
#The average number of ratings given to the beers
beer.groupby('beer_beerid').count().review_overall.mean()

145.06755424063115

In [24]:
#The average number of ratings given by the users
beer.groupby('review_profilename').count().review_overall.mean()

15.128143158327763

**Model Building**

In [0]:
#Split into train-test
from sklearn.model_selection import train_test_split
train, test = train_test_split(beer, test_size=0.30, random_state=31)

In [26]:
print(train.shape)
print(test.shape)

(205937, 3)
(88260, 3)


In [0]:
# pivot ratings into beer features
df_beer_features = train.pivot(index='review_profilename',columns='beer_beerid',values='review_overall').fillna(0)

In [28]:
df_beer_features.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01Ryan10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04101Brewer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0beerguy0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Make copy of train and test
dummy_train = train.copy()
dummy_test = test.copy()

In [30]:
print(dummy_train.head())
dummy_train.shape

        beer_beerid review_profilename  review_overall
162658         5948    TheEclecticGent             4.5
217111         2961          Franchise             4.0
29063         54089              Jayli             3.0
24752          3558          terrabyte             3.5
312729        20768      darklordlager             4.0


(205937, 3)

In [0]:
#Make all non reviewed cell as 1 and rest all 0 for train and vice versa for test.
dummy_train['review_overall'] = dummy_train['review_overall'].apply(lambda x: 0 if x >= 1 else 1)
dummy_test['review_overall'] = dummy_test['review_overall'].apply(lambda x: 1 if x >=1 else 0)

In [0]:
# The beer not rated by user is marked as 1 for prediction. 
dummy_train = dummy_train.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
).fillna(1)

# The beer not rated by user is marked as 0 for evaluation. 
dummy_test = dummy_test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
).fillna(0)

In [33]:
dummy_train.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01Ryan10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
04101Brewer,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
05Harley,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0beerguy0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [34]:
dummy_test.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0tt0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100floods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Pivot beer features
beer_features = train.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
)

In [36]:
beer_features.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01Ryan10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04101Brewer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05Harley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0beerguy0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#Take df_subtracted which is mean of nan values subtracted from original df
mean = np.nanmean(beer_features, axis=1)
df_subtracted = (beer_features.T-mean).T
df_subtracted=df_subtracted.fillna(0)
df_subtracted.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01Ryan10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04101Brewer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0beerguy0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Create cosine Similarity matrix
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(df_subtracted, metric='cosine')

In [39]:
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
#Remove all negative recommondation
user_correlation[user_correlation<0]=0
user_correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**Prediction for user based**

In [41]:
#Prediction for user based model
user_predicted_ratings = np.dot(user_correlation, beer_features.fillna(0))
user_predicted_ratings

array([[1.18210979, 2.39586718, 2.00712931, ..., 1.30620554, 2.19226756,
        1.58426839],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.56729534, 5.22367115, 2.53076101, ..., 0.44452145, 0.71906464,
        2.19279431],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [42]:
user_predicted_ratings.shape

(17202, 2028)

In [43]:
#multiply user prediction with dummy train to get final ratings
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,1.182110,2.395867,2.007129,2.383969,0.117704,1.158166,2.348213,1.834791,5.900556,3.069009,6.320933,11.004264,1.941328,7.726463,0.851823,3.626533,2.460885,5.211831,5.729038,6.421275,2.249000,1.416910,0.452228,7.275238,2.234638,0.608799,1.056725,0.633652,1.720428,0.258660,0.539217,2.916656,1.494359,3.030484,9.209862,2.060899,6.905795,9.358838,1.029690,1.771866,...,1.201429,0.628539,1.731964,1.644536,1.608598,2.199801,2.973070,0.751025,1.431912,1.406231,1.280014,3.704292,0.746748,1.205332,0.922172,1.202831,1.187052,0.739329,1.605611,1.479110,1.543608,1.860918,1.896229,2.089652,1.986690,2.640293,2.868229,1.501765,0.721702,1.666966,0.807601,2.263625,6.130013,0.537544,0.302546,0.470864,1.041869,1.306206,2.192268,1.584268
01Ryan10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
04101Brewer,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
05Harley,1.431752,4.224999,2.486638,5.221106,0.468807,1.552946,4.277244,2.620628,12.121386,4.876519,9.005713,14.816606,2.704942,16.627088,0.981458,2.600305,4.075711,9.580368,8.816293,7.739463,3.139157,1.956615,1.257172,11.004560,2.800782,1.221389,2.244758,1.119294,0.963324,1.084311,2.059314,2.493850,0.759706,5.758014,17.675394,1.583309,11.486912,17.300144,1.695604,4.958104,...,3.078421,1.736015,3.476363,3.698615,2.424916,2.922637,4.553756,1.336970,3.610062,4.256438,5.178665,6.219080,2.377914,1.418476,1.944194,0.423755,1.744399,2.239807,2.848915,2.544137,3.263559,4.154128,4.044856,2.377247,1.506155,4.559712,5.468866,5.127400,1.942665,3.779781,2.092030,2.979623,7.615892,1.598967,2.103561,2.601054,2.245057,0.755234,4.726285,2.286082
0beerguy0,0.667724,2.713345,1.434856,2.287425,1.973061,0.338014,2.287190,0.928182,2.797073,3.126014,4.986815,3.853786,2.049074,2.107295,0.834249,1.019267,1.624108,2.914281,5.418280,3.507528,2.380354,0.735384,0.758893,7.418447,1.573399,0.706402,2.033975,1.008003,0.654919,0.674027,0.860885,1.267181,0.875014,1.402011,4.832166,0.804838,4.682317,4.634420,1.001609,0.858193,...,0.451129,0.878495,0.207694,0.718884,1.327884,0.159696,0.373151,0.814108,0.144863,1.929230,0.304834,0.425512,1.098995,0.173309,0.063942,0.163250,0.174419,0.099903,0.008093,0.270553,1.617737,0.869063,1.245599,0.3

In [44]:
#Test model with one of the user with top 5 recommended beers
user_final_rating.iloc[3].sort_values(ascending=False)[0:5]

beer_beerid
412      30.017483
17112    28.251250
4083     27.088169
19960    26.874796
1904     24.497512
Name: 05Harley, dtype: float64

In [45]:
#Use transponse of beer features for item based model
beer_item_features = train.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
).T

beer_item_features.head()

review_profilename,0110x011,01Ryan10,04101Brewer,05Harley,0beerguy0,0runkp0s,0tt0,1000Bottles,100floods,1050Sudz,108Dragons,1099,1121987,11millsown113,11osixBrew,11thFloorBrewing,12ouncecurls,12vUnion,13aphomet,13smurrf,160Shillings,1759Girl,18alpha,1Adam12,1after909,1fastz28,1inamill,1morebeer,1noa,1quiks10,1santore,1whiskey,20ozmonkey,21mmer,220emaple,2378GCGTG,23fyerfyter,24Beer92,28Rock,2BDChicago,...,zfjohnson,zhanson88,zhooker,zic,ziggy13,zimm421,zimmerbloke,zipperergj,zjungleist,zloew,zminor,zms101,znarf,znichols,zodiacv1,zodiak3000,zooalbert,zooga,zook74,zoolzoo,zornbeast,zoso1967,zoso493,zplug123,zrab11,zseeanz,zsk,ztaylor1,ztprez,zuccard,zuffenhausen,zuggy9,zulufactor,zwalk8,zymrgy,zymurgy4all,zymurgywhiz,zythus,zyzygy,zzajjber
beer_beerid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Take subtracted df which is mean of nan values subtracted from original df
mean = np.nanmean(beer_item_features, axis=1)
df_item_based_subtracted = (beer_item_features.T-mean).T

In [47]:
df_item_based_subtracted.head()

review_profilename,0110x011,01Ryan10,04101Brewer,05Harley,0beerguy0,0runkp0s,0tt0,1000Bottles,100floods,1050Sudz,108Dragons,1099,1121987,11millsown113,11osixBrew,11thFloorBrewing,12ouncecurls,12vUnion,13aphomet,13smurrf,160Shillings,1759Girl,18alpha,1Adam12,1after909,1fastz28,1inamill,1morebeer,1noa,1quiks10,1santore,1whiskey,20ozmonkey,21mmer,220emaple,2378GCGTG,23fyerfyter,24Beer92,28Rock,2BDChicago,...,zfjohnson,zhanson88,zhooker,zic,ziggy13,zimm421,zimmerbloke,zipperergj,zjungleist,zloew,zminor,zms101,znarf,znichols,zodiacv1,zodiak3000,zooalbert,zooga,zook74,zoolzoo,zornbeast,zoso1967,zoso493,zplug123,zrab11,zseeanz,zsk,ztaylor1,ztprez,zuccard,zuffenhausen,zuggy9,zulufactor,zwalk8,zymrgy,zymurgy4all,zymurgywhiz,zythus,zyzygy,zzajjber
beer_beerid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.29096,-0.209040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.778986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.112179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#Get cosine  Item Similarity Matrix
from sklearn.metrics.pairwise import pairwise_distances

# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_item_based_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[ 1.00000000e+00  8.29686245e-03  2.31720924e-03 ...  7.95984223e-05
   1.41341239e-02 -3.28010645e-03]
 [ 8.29686245e-03  1.00000000e+00  1.74066299e-03 ... -1.34282657e-02
   2.26833774e-03  4.21673842e-03]
 [ 2.31720924e-03  1.74066299e-03  1.00000000e+00 ...  0.00000000e+00
   8.66663720e-04  2.16803081e-02]
 ...
 [ 7.95984223e-05 -1.34282657e-02  0.00000000e+00 ...  1.00000000e+00
   9.49920731e-03  2.53744976e-02]
 [ 1.41341239e-02  2.26833774e-03  8.66663720e-04 ...  9.49920731e-03
   1.00000000e+00 -1.74653081e-02]
 [-3.28010645e-03  4.21673842e-03  2.16803081e-02 ...  2.53744976e-02
  -1.74653081e-02  1.00000000e+00]]


In [49]:
#Take only positive values to avoid negative recommedation
item_correlation[item_correlation<0]=0
item_correlation

array([[1.00000000e+00, 8.29686245e-03, 2.31720924e-03, ...,
        7.95984223e-05, 1.41341239e-02, 0.00000000e+00],
       [8.29686245e-03, 1.00000000e+00, 1.74066299e-03, ...,
        0.00000000e+00, 2.26833774e-03, 4.21673842e-03],
       [2.31720924e-03, 1.74066299e-03, 1.00000000e+00, ...,
        0.00000000e+00, 8.66663720e-04, 2.16803081e-02],
       ...,
       [7.95984223e-05, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 9.49920731e-03, 2.53744976e-02],
       [1.41341239e-02, 2.26833774e-03, 8.66663720e-04, ...,
        9.49920731e-03, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.21673842e-03, 2.16803081e-02, ...,
        2.53744976e-02, 0.00000000e+00, 1.00000000e+00]])

**Prediction for Item based**

In [50]:
#Find predicted ratings with dot product
item_predicted_ratings = np.dot((df_item_based_subtracted.fillna(0).T),item_correlation)
item_predicted_ratings

array([[ 0.00242126,  0.01702397,  0.00016852, ...,  0.05174728,
         0.05323234,  0.02654053],
       [ 0.        ,  0.        ,  0.00491195, ...,  0.02092873,
         0.01038159,  0.        ],
       [ 0.        , -0.00269408, -0.00105854, ...,  0.        ,
         0.        , -0.00526907],
       ...,
       [ 0.03007527,  0.        ,  0.01807933, ...,  0.00980537,
         0.02202895,  0.01431684],
       [ 0.00020038,  0.01236403,  0.00376191, ...,  0.        ,
         0.0016721 ,  0.00658889],
       [ 0.        ,  0.00919882,  0.01291677, ...,  0.        ,
         0.        ,  0.        ]])

In [51]:
item_predicted_ratings.shape

(17202, 2028)

In [52]:
dummy_train.shape

(17202, 2028)

In [53]:
#Multiply predicted ratings with dummy to get final ratings
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.002421,0.017024,0.000169,0.025937,0.086133,0.029311,0.027652,0.006012,0.029544,-0.055032,0.032519,0.020019,0.003791,-0.005463,-0.033999,0.017237,0.057426,-0.009125,0.006879,0.022533,0.031583,0.012884,0.019828,0.027103,0.038969,0.026665,0.002087,0.053441,0.027844,-0.000542,0.012297,0.067624,0.008852,0.026388,-0.038122,0.024165,0.022035,0.045719,0.021910,0.009284,...,0.010582,-0.025742,0.188787,-0.071663,0.010501,0.039792,0.002983,0.020206,0.041724,0.008358,0.005151,0.114061,0.032940,0.046133,0.028465,0.024846,0.013956,-0.002058,0.043587,-0.013037,0.020063,0.100005,0.030215,0.009933,0.009634,0.063532,-0.043425,0.060123,-0.022955,0.006001,0.095985,0.025236,0.058975,0.014241,-0.010926,0.019443,0.014123,0.051747,0.053232,0.026541
01Ryan10,0.000000,0.000000,0.004912,0.000000,0.000000,0.000000,0.007903,0.000000,0.003025,0.000000,0.000000,0.000000,0.011694,0.010001,0.004363,0.011975,0.000000,0.025232,0.002515,0.003690,0.009378,0.000000,0.006546,0.000000,0.004766,0.000000,0.008922,0.000000,0.000000,0.000000,0.000000,0.008841,0.000842,0.000000,0.002574,0.000000,0.000000,0.000000,0.002199,0.000000,...,0.014409,0.012955,0.103620,0.001667,0.000000,0.000000,0.010353,0.001505,0.000000,0.008220,0.000000,0.041152,0.000000,0.000000,0.000000,0.004419,0.000000,0.038650,0.014481,0.027895,0.004811,0.006362,0.000000,0.023681,0.040111,0.007207,0.000000,0.019306,0.000000,0.024142,0.045935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020929,0.010382,0.000000
04101Brewer,0.000000,-0.002694,-0.001059,-0.001922,0.000000,0.000000,0.000000,0.000000,-0.003471,-0.008917,0.000000,-0.000116,-0.001027,0.000000,0.000000,0.000000,0.000000,-0.002399,-0.001366,-0.001316,0.000000,0.000000,0.000000,-0.000324,-0.006915,0.000000,-0.000274,-0.000899,-0.016055,0.000000,0.000000,-0.005772,-0.000133,-0.001253,-0.000349,0.000000,-0.002255,-0.000248,0.000000,-0.002555,...,-0.000672,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001848,0.000000,0.000000,-0.001529,-0.013142,-0.003178,0.000000,0.000000,-0.014323,0.000000,-0.003923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003257,-0.000406,-0.001960,-0.001289,-0.007047,-0.001916,0.000000,0.000000,-0.005010,-0.000922,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005269
05Harley,-0.021452,-0.001032,0.021830,0.018759,-0.012978,-0.011302,-0.039518,-0.028967,-0.014290,-0.024641,-0.026908,-0.029531,-0.061729,0.014355,-0.011729,-0.014991,-0.003377,-0.063547,-0.003164,-0.007927,-0.043044,0.005254,0.009032,0.001578,-0.003193,-0.003763,-0.018667,-0.019631,0.006628,-0.004616,-0.014388,-0.017743,-0.014514,-0.002829,-0.000687,-0.023143,-0.005662,-0.016899,0.010031,0.001442,...,0.008257,-0.045088,0.004450,0.002361,0.001111,0.028902,-0.019819,-0.003536,-0.043470,0.003292,0.004723,-0.010763,0.015484,-0.001121,0.002305,-0.023949,-0.002926,-0.025227,-0.015659,0.000790,-0.013901,0.037323,-0.005683,0.011285,0.002367,-0.027251,0.023950,0.001635,-0.004074,-0.075510,-0.015469,0.006564,0.024686,0.017192,0.001283,-0.020866,-0.009495,-0.001501,-0.028914,0.023313
0beerguy0,0.007324,-0.010586,0.010634,-0.087198,-0.043827,0.016354,-0.032290,-0.004852,-0.002816,0.017540,-0.005776,-0.012957,-0.027613,-0.014849,0.003310,-0.000727,0.022739,0.010675,-0.013837,-0.030460,-0.009479,-0.023214,-0.025580,-0.004416,0.003863,-0.017132,0.016888,-0.033425,-0.016340,0.001538,-0.001578,0.002567,-0.058644,-0.006533,0.006288,-0.080766,-0.029256,0.000638,0.002148,0.000916,...,-0.002828,0.000000,-0.005732,-0.002491,0.000000,-0.002157,-0.000988,0.010137,-0.011619,0.00

In [71]:
#Model test for item based
item_final_rating.iloc[3].sort_values(ascending=False)[0:5]

beer_beerid
48194    0.088989
4878     0.070944
37966    0.068034
62328    0.066027
16439    0.063243
Name: 05Harley, dtype: float64

In [57]:
item_final_rating.head(10)

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,2.421263e-03,0.017024,0.000169,0.025937,0.086133,0.029311,0.027652,0.006012,0.029544,-0.055032,0.032519,0.020019,0.003791,-0.005463,-0.033999,0.017237,0.057426,-0.009125,0.006879,0.022533,0.031583,0.012884,0.019828,0.027103,0.038969,0.026665,0.002087,0.053441,0.027844,-0.000542,0.012297,0.067624,0.008852,0.026388,-0.038122,0.024165,0.022035,0.045719,0.021910,0.009284,...,0.010582,-0.025742,0.188787,-0.071663,0.010501,0.039792,0.002983,0.020206,0.041724,0.008358,0.005151,0.114061,0.032940,0.046133,0.028465,0.024846,0.013956,-0.002058,0.043587,-0.013037,0.020063,0.100005,0.030215,0.009933,0.009634,0.063532,-0.043425,0.060123,-0.022955,0.006001,0.095985,0.025236,0.058975,0.014241,-0.010926,0.019443,0.014123,0.051747,0.053232,0.026541
01Ryan10,0.000000e+00,0.000000,0.004912,0.000000,0.000000,0.000000,0.007903,0.000000,0.003025,0.000000,0.000000,0.000000,0.011694,0.010001,0.004363,0.011975,0.000000,0.025232,0.002515,0.003690,0.009378,0.000000,0.006546,0.000000,0.004766,0.000000,0.008922,0.000000,0.000000,0.000000,0.000000,0.008841,0.000842,0.000000,0.002574,0.000000,0.000000,0.000000,0.002199,0.000000,...,0.014409,0.012955,0.103620,0.001667,0.000000,0.000000,0.010353,0.001505,0.000000,0.008220,0.000000,0.041152,0.000000,0.000000,0.000000,0.004419,0.000000,0.038650,0.014481,0.027895,0.004811,0.006362,0.000000,0.023681,0.040111,0.007207,0.000000,0.019306,0.000000,0.024142,0.045935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020929,0.010382,0.000000
04101Brewer,0.000000e+00,-0.002694,-0.001059,-0.001922,0.000000,0.000000,0.000000,0.000000,-0.003471,-0.008917,0.000000,-0.000116,-0.001027,0.000000,0.000000,0.000000,0.000000,-0.002399,-0.001366,-0.001316,0.000000,0.000000,0.000000,-0.000324,-0.006915,0.000000,-0.000274,-0.000899,-0.016055,0.000000,0.000000,-0.005772,-0.000133,-0.001253,-0.000349,0.000000,-0.002255,-0.000248,0.000000,-0.002555,...,-0.000672,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001848,0.000000,0.000000,-0.001529,-0.013142,-0.003178,0.000000,0.000000,-0.014323,0.000000,-0.003923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003257,-0.000406,-0.001960,-0.001289,-0.007047,-0.001916,0.000000,0.000000,-0.005010,-0.000922,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005269
05Harley,-2.145208e-02,-0.001032,0.021830,0.018759,-0.012978,-0.011302,-0.039518,-0.028967,-0.014290,-0.024641,-0.026908,-0.029531,-0.061729,0.014355,-0.011729,-0.014991,-0.003377,-0.063547,-0.003164,-0.007927,-0.043044,0.005254,0.009032,0.001578,-0.003193,-0.003763,-0.018667,-0.019631,0.006628,-0.004616,-0.014388,-0.017743,-0.014514,-0.002829,-0.000687,-0.023143,-0.005662,-0.016899,0.010031,0.001442,...,0.008257,-0.045088,0.004450,0.002361,0.001111,0.028902,-0.019819,-0.003536,-0.043470,0.003292,0.004723,-0.010763,0.015484,-0.001121,0.002305,-0.023949,-0.002926,-0.025227,-0.015659,0.000790,-0.013901,0.037323,-0.005683,0.011285,0.002367,-0.027251,0.023950,0.001635,-0.004074,-0.075510,-0.015469,0.006564,0.024686,0.017192,0.001283,-0.020866,-0.009495,-0.001501,-0.028914,0.023313
0beerguy0,7.323503e-03,-0.010586,0.010634,-0.087198,-0.043827,0.016354,-0.032290,-0.004852,-0.002816,0.017540,-0.005776,-0.012957,-0.027613,-0.014849,0.003310,-0.000727,0.022739,0.010675,-0.013837,-0.030460,-0.009479,-0.023214,-0.025580,-0.004416,0.003863,-0.017132,0.016888,-0.033425,-0.016340,0.001538,-0.001578,0.002567,-0.058644,-0.006533,0.006288,-0.080766,-0.029256,0.000638,0.002148,0.000916,...,-0.002828,0.000000,-0.005732,-0.002491,0.000000,-0.002157,-0.000988,0.0

In [58]:
user_final_rating.head(10)

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,1.182110,2.395867,2.007129,2.383969,0.117704,1.158166,2.348213,1.834791,5.900556,3.069009,6.320933,11.004264,1.941328,7.726463,0.851823,3.626533,2.460885,5.211831,5.729038,6.421275,2.249000,1.416910,0.452228,7.275238,2.234638,0.608799,1.056725,0.633652,1.720428,0.258660,0.539217,2.916656,1.494359,3.030484,9.209862,2.060899,6.905795,9.358838,1.029690,1.771866,...,1.201429,0.628539,1.731964,1.644536,1.608598,2.199801,2.973070,0.751025,1.431912,1.406231,1.280014,3.704292,0.746748,1.205332,0.922172,1.202831,1.187052,0.739329,1.605611,1.479110,1.543608,1.860918,1.896229,2.089652,1.986690,2.640293,2.868229,1.501765,0.721702,1.666966,0.807601,2.263625,6.130013,0.537544,0.302546,0.470864,1.041869,1.306206,2.192268,1.584268
01Ryan10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
04101Brewer,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
05Harley,1.431752,4.224999,2.486638,5.221106,0.468807,1.552946,4.277244,2.620628,12.121386,4.876519,9.005713,14.816606,2.704942,16.627088,0.981458,2.600305,4.075711,9.580368,8.816293,7.739463,3.139157,1.956615,1.257172,11.004560,2.800782,1.221389,2.244758,1.119294,0.963324,1.084311,2.059314,2.493850,0.759706,5.758014,17.675394,1.583309,11.486912,17.300144,1.695604,4.958104,...,3.078421,1.736015,3.476363,3.698615,2.424916,2.922637,4.553756,1.336970,3.610062,4.256438,5.178665,6.219080,2.377914,1.418476,1.944194,0.423755,1.744399,2.239807,2.848915,2.544137,3.263559,4.154128,4.044856,2.377247,1.506155,4.559712,5.468866,5.127400,1.942665,3.779781,2.092030,2.979623,7.615892,1.598967,2.103561,2.601054,2.245057,0.755234,4.726285,2.286082
0beerguy0,0.667724,2.713345,1.434856,2.287425,1.973061,0.338014,2.287190,0.928182,2.797073,3.126014,4.986815,3.853786,2.049074,2.107295,0.834249,1.019267,1.624108,2.914281,5.418280,3.507528,2.380354,0.735384,0.758893,7.418447,1.573399,0.706402,2.033975,1.008003,0.654919,0.674027,0.860885,1.267181,0.875014,1.402011,4.832166,0.804838,4.682317,4.634420,1.001609,0.858193,...,0.451129,0.878495,0.207694,0.718884,1.327884,0.159696,0.373151,0.814108,0.144863,1.929230,0.304834,0.425512,1.098995,0.173309,0.063942,0.163250,0.174419,0.099903,0.008093,0.270553,1.617737,0.869063,1.245599,0.3

In [99]:
# top 5 beers that you would recommend to the users 'cokes', 'genog' and 'giblet' using both the models.
print(user_final_rating.loc['cokes'].sort_values(ascending=False)[0:5])
print(item_final_rating.loc['cokes'].sort_values(ascending=False)[0:5])
print(user_final_rating.loc['genog'].sort_values(ascending=False)[0:5])
print(item_final_rating.loc['genog'].sort_values(ascending=False)[0:5])
print(user_final_rating.loc['giblet'].sort_values(ascending=False)[0:5])
item_final_rating.loc['giblet'].sort_values(ascending=False)[0:5]

beer_beerid
412     42.048229
1093    38.781419
1708    36.911971
4083    36.850615
34      36.494164
Name: cokes, dtype: float64
beer_beerid
356      0.712988
1713     0.640622
38149    0.631892
34363    0.599630
31323    0.594910
Name: cokes, dtype: float64
beer_beerid
2093    33.613647
1093    32.375077
1013    29.789423
34      29.485823
92      29.386374
Name: genog, dtype: float64
beer_beerid
40956    0.246236
45368    0.236574
1959     0.226124
27476    0.225972
27143    0.203829
Name: genog, dtype: float64
beer_beerid
2093    63.411020
104     62.002390
412     61.935268
1093    56.511968
34      53.872070
Name: giblet, dtype: float64


beer_beerid
40798    0.080821
48434    0.061476
31613    0.060388
3279     0.059108
67040    0.056448
Name: giblet, dtype: float64

**Comparing the performance of the two models using test data**

In [56]:
test_beer_features = test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
)
mean = np.nanmean(test_beer_features, axis=1)
test_df_subtracted = (test_beer_features.T-mean).T

# User Similarity Matrix
test_user_correlation = 1 - pairwise_distances(test_df_subtracted.fillna(0), metric='cosine')
test_user_correlation[np.isnan(test_user_correlation)] = 0
print(test_user_correlation)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
test_user_correlation[test_user_correlation<0]=0
test_user_predicted_ratings = np.dot(test_user_correlation, test_beer_features.fillna(0))
test_user_predicted_ratings

array([[0.25759248, 1.03114226, 0.42726579, ..., 0.16461519, 0.60528718,
        0.74483352],
       [0.17607375, 0.59829136, 0.41797999, ..., 0.71338242, 0.        ,
        0.26969114],
       [0.35672692, 0.63287196, 0.06750944, ..., 0.6041211 , 0.        ,
        0.07923394],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.53947716, 1.81844717, 0.45549682, ..., 0.00343226, 0.73337831,
        1.22483534],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [0]:
test_user_final_rating = np.multiply(test_user_predicted_ratings,dummy_test)

In [0]:
test_user_final_rating.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0tt0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100floods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.902635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#Scaling to get original ratings in data
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = test_user_final_rating.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=([0,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(copy=True,
             feature_range=[0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [0]:
#Finding RMSE for user based
test_ = user_based_test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
)
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))
rmse = (sum(sum((test_ - y )**2))/total_non_nan)**0.5
print(rmse)

3.6324965406761134


In [0]:
test_beer_features = user_based_test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
).T

mean = np.nanmean(test_beer_features, axis=1)
test_df_item_subtracted = (test_beer_features.T-mean).T

test_item_correlation = 1 - pairwise_distances(test_df_item_subtracted.fillna(0), metric='cosine')
test_item_correlation[np.isnan(test_item_correlation)] = 0
test_item_correlation[test_item_correlation<0]=0

In [0]:
test_item_predicted_ratings = (np.dot(test_item_correlation, test_beer_features.fillna(0))).T
test_item_final_rating = np.multiply(test_item_predicted_ratings,dummy_test)
test_item_final_rating.head()

beer_beerid,5,6,7,10,14,15,17,19,30,31,33,34,36,39,54,55,58,59,61,63,65,67,69,73,74,78,79,80,81,82,83,85,86,87,88,89,90,92,93,96,...,65200,65325,65862,66036,66037,66281,66436,67040,67044,67106,67156,67219,67230,67262,67267,68008,68400,68746,68772,68916,68958,68960,69070,69363,69407,70102,70910,71405,71758,71774,71919,71969,72138,73131,74123,74272,74323,74390,74986,75086
review_profilename,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0110x011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Harley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0tt0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100floods,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.904903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
test_ = user_based_test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
)

In [0]:
#Scaling to get original ratings
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = test_item_final_rating.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=([0,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]))
print(scaler.fit(X))
y = (scaler.transform(X))


test_ = user_based_test.pivot(
    index='review_profilename',
    columns='beer_beerid',
    values='review_overall'
)

# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

MinMaxScaler(copy=True,
             feature_range=[0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])


In [0]:
#RMSE score for item based model
rmse = (sum(sum((test_ - y )**2))/total_non_nan)**0.5
print(rmse)

3.645549453427868


*Since using user based has less value of RMSE, it will perform better than item based.*